<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V2_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages and Importing libraries**

In [ ]:
!pip install gekko
!pip install ipynb
#!pip install modin[dask]
import numpy as np
import time
import math
import datetime
 
from random import sample
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
#import modin.pandas as pd
import random 
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *

     |████████████████████████████████| 10.8MB 9.0MB/s 
Mounted at /content/drive


**SAWMILL CLASS**

In [9]:
DISTANCE_MODE = 1     #Harvesting mode that reach closest locations
DENSITY_MODE = 2      #Harvesting mode that reach more dense locations
HAULING_COST = 0.71   #Hauling cost per unit
MAX_RADIO = 60        #Maximum Radio distance from mills
RADIO_MILL = 1        #Radio distance from mills 
RADIO_REFINERY = 100        #Radio distance from mills 
RESIDUES_LEFT = 0.3   #Percentage residues left in the ground
TRANSP_COST = 0.054   #Transportation cost per unit
WILLING_FACTOR = 0.45 #Willingness to harvest factor
 
 
class Inventory:
  Total_Harvested = []
  Total_Residues = []
  
  #Total_Harvested = pd.DataFrame({'Unique_ID' : [],'Harvested':[]}) #initialize Total Harvested variable (for all the layers)
  #Total_Residues = pd.DataFrame({'Unique_ID' : [],'SumOfTotal':[]}) #Initialize Total Residues variable (for all the layers)
  
  def __init__(self, Av, Dist, Wt=-1, Pt=-1,Name ="", Willingness = 0.45):
    self.AcumResidues = []  #Residues generated by harvesting
    self.Available = list()
    self.Available.append(Av.copy()) #Dataframe that contains Available wood Inventory at diferent time t
    self.Available[0].loc[:,('SumOfTotal')] = self.Available[0].loc[:,('SumOfTotal')] * Willingness
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the mill to be harvested
    self.By_Mill = [] #Quantity harvested by each mills in each hexagon
    self.Candidates=[] #Candidate location for biorefinery
    self.Dist = Dist #Matrix Distance from any Mills or candidates cells to other cells
    self.Dist_Mill = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Dist_Refi = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Factor = [] #Dataframe that contains the Distance to the mill to be harvested

    self.Harvested = [] #Ammount of harvested wood
    self.Harvested_Res = []  #Residues Harvested
    self.Name = Name
    self.ProdType = Pt #type of product generated (pulp or saw)
    self.RefCost = [] #Matrix Distance from any Mills and candidates cells to other cells
    self.Residues = []  #Residues generated by harvesting
    self.Residues_Left = 0.30 #portion of residues left in the ground and not used for the biorefinery
    self.Willingness = Willingness #portion of timber available for residues.
    self.WoodType = Wt #type of wood needed (softwood, hardwood)
    self.WoodType = Wt #type of wood needed (softwood, hardwood)
      
    #initialize Harvested list similar to available 
    
    Id_li = list(self.Available[0].loc[:,("Unique_ID")])
    r1 = len(self.Available[0])
    Ha_li =[0]*r1
    Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
    self.Harvested.append(pd.DataFrame(Data).copy())
    Data = {'Unique_ID':Id_li,'Growth':Ha_li}
    self.Growth =pd.DataFrame(Data).copy()    
    Data = {'Unique_ID':Id_li}
    self.By_Mill =pd.DataFrame(Data).copy()
    Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
    self.Harvested_Res.append(pd.DataFrame(Data).copy())

    
  def Harvest_All_Mills(self,t,k,mode = DISTANCE_MODE):
  #Harvest k mills at year t for the first stage
    self.Available.append(self.Available[t].copy())
    for i in range(k):
      print('Cosechando Sawmill:',i)
      self.Harvest_Mill(t,i,mode)
 
    self.Harvested.append(self.Harvested[t].copy()) 
    self.Harvested[t+1].loc[:,('Harvested')] = 0
    self.Residues.append(self.Harvested[t].loc[:,('Unique_ID','Harvested')].copy())
    self.Residues[t].loc[:,('Harvested')] = round(self.Residues[t].loc[:,('Harvested')]*(1-RESIDUES_LEFT)) #substract the portion of residues left in the ground
    self.Residues[t].loc[:,('Harvested')] = round(self.Residues[t].loc[:,('Harvested')]*0.01814*0.55) #transforming CF to Dry Tons
    print('Residues:',np.sum(self.Residues[t].Harvested))
  
  def Harvest_All_Mills_All_Years(self,Y,mode=DISTANCE_MODE):
    N_Mills = len(self.Mills)
    now = datetime.datetime.now()
    for y in range(YEARS):
      self.Harvest_All_Mills(y,2,mode)
      self.Add_Growth(y) 
    self.Residues.append(self.Residues[0].loc[:,('Unique_ID','Harvested')].copy())
    self.Residues[y].loc[:,('Harvested')] = 0
    for y in range(Y): 
      self.Harvested[y].to_csv('/content/drive/My Drive/Colab Notebooks/Results/'+str(mode)+self.Name+'_'+str(now.day)+'_'+' year'+str(y)+'.csv')
      self.By_Mill.to_csv('/content/drive/My Drive/Colab Notebooks/Results/'+str(mode)+self.Name+'_ByMills'+'_'+str(now.day)+'_'+' year'+str(y)+'.csv')
    
 
  def Growth_Update(self,Data):
    N = len(self.Growth)
    if self.ProdType == 'Sawmill':
      cl = 1
    elif self.ProdType == 'Pulpmill':
      cl =2
    Data = Data[Data['CLASS'] == cl]
    for i in range(N):    
      Id = self.Growth.loc[i,("Unique_ID")]
      Row = Data[Data['Unique_ID'] == Id]
      if not Row.empty:
        if self.WoodType == 'H':
          self.Growth.loc[i,('Growth')] = Row.iloc[0,4] + Row.iloc[0,5] + 0.33 * Row.iloc[0,8]
        if self.WoodType == 'S':
          self.Growth.loc[i,('Growth')] = Row.iloc[0,6] + Row.iloc[0,7] + 0.67 * Row.iloc[0,8]
    
 
  def Cell_Less(self, Id_Mill, t, Radio=  RADIO_MILL):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #Look for in Dist matrix cell with the ID and closer than Radio
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
 
    for j in range(len(Ans)):
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal.copy()))
    Ans["AVAIL"] = Disp 
    print('Cell_Less returns:\n')
    Ans.head(20)
    return(Ans)
 
 
  def Cell_Less_Res(self, Id_Mill, t, Radio=50):
    #Given Time, Radio and a Mill ID this function returns a Dataframe of cells with Distance and Forest Residues available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
 
    for j in range(len(Ans)):                                                                         
      if self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].Harvested))
    Ans["AVAIL"] = Disp 
    return(Ans)
 
 
  def Disp_Radio(self, Id_Mill, t, Radio=RADIO_MILL):
    #Given a Time and Radio and a Mill ID this function returns a list of Inventory Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)
  
 
  def Num_Cell(self, Id_Mill, Radio=RADIO_MILL):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    return(len(Ans))
  
  def Mills_Update(self,M):
    i1 = (M['s_h'] == self.WoodType)&(M['plant'] == self.ProdType)
    self.Mills = M.loc[i1,:]
    #self.Mills.reset_index(drop=True, inplace=True)
    
    Num = len(self.Mills)
    ind = [i for i in range(Num)]
    self.Mills.index = ind
    
 
  def Disp_Radio_Res(self, Id_Mill, t, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Residues Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= RADIO_REFINERY)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    #update index from 0 to rows-1
    Disp =[]
    for j in range(len(Ans)):  
      #print(self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')])                                                   
      if self.AcumResidues[t][self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.AcumResidues[t][self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)
 
 
   
  def Optimal_Location_Total(self,Y,NCan = 0):
  #run Ncan locations from year 0 to year 'y'
    if (NCan == 0):
      Cans = len(self.Candidates)
    else:
      Cans = NCan
    self.RefCost = [0]*Cans
 
    for c in range(Cans):
      self.AcumResidues = []
      self.AcumResidues.append(self.Residues[0].copy())
      self.AcumResidues[0].columns=["Unique_ID", "SumOfTotal"]
      self.AcumResidues[0].loc[:,('SumOfTotal')] = 0
        
      for i in range(Y):
        self.AcumResidues.append(self.AcumResidues[i].copy())
        self.AcumResidues[i+1].columns=["Unique_ID", "SumOfTotal"]
        self.AcumResidues[i+1]['SumOfTotal'] =self.AcumResidues[i+1]['SumOfTotal'] + self.Residues[i]['Harvested']
        self.Harvest_Refinery(i,c)
        self.Harvested_Res.append(self.Harvested_Res[i].copy())     
        self.Harvested_Res[i+1].loc[:,('Harvested')] = 0
 

  def Add_Growth(self,t):
    #increase the available inventory of period t
    self.Available[t].loc[:,("SumOfTotal")] = self.Available[t].loc[:,("SumOfTotal")]+self.Growth['Growth']
  
 
  def Initialize_Totals(self,Y):
    self.__class__.Total_Harvested = []
    self.__class__.Total_Residues = []
    for i in range(Y+1):
      self.__class__.Total_Harvested.append(pd.DataFrame({'Unique_ID' : [],'Harvested':[]})) #initialize Total Harvested variable (for all the layers)
      self.__class__.Total_Residues.append(pd.DataFrame({'Unique_ID' : [],'Harvested':[]})) #Initialize Total Residues variable (for all the layers)
  
  def Total_Harvested_Update(self,Y):
    for i in range(Y):
      self.Harvested[Y].loc[:,('Harvested')] = self.Harvested[Y].loc[:,('Harvested')] + self.Harvested[i].loc[:,('Harvested')]
 
    for i in range(Y+1):
      c = pd.concat([self.__class__.Total_Harvested[i], self.Harvested[i]]).groupby(['Unique_ID']).sum().reset_index() #updating Total Harvested
      self.__class__.Total_Harvested[i] = c.copy()
 
  def Total_Residues_Update(self,Y):
    for i in range(Y):
      self.Residues[Y].loc[:,('Harvested')] = self.Residues[Y].loc[:,('Harvested')] + self.Residues[i].loc[:,('Harvested')]
 
    for i in range(Y+1):
      c = pd.concat([self.__class__.Total_Residues[i], self.Residues[i]]).groupby(['Unique_ID']).sum().reset_index() #updating Total Harvested
      self.__class__.Total_Residues[i] = c.copy()
 
 
  def Total_Available(self, Id_Mill, Radio=RADIO_MILL):
    #Given a Radio and a Mill ID this function returns a the total inventory Available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal.copy()))
    Total = np.sum(Disp)
    return(Total)
 
 
  def Harvest_Mill(self,t, j,mode=DISTANCE_MODE):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3                        #IPOPT solver
    m.options.IMODE = 3                         #steady state optimization
    Dem = self.Mills.loc[j,('Demand')]
    Id1 =self.Mills.loc[j,('Unique_ID')]        #look for mill in position j
    Name_Mill =self.Mills.loc[j,('Name')]       #look for mill in position j
    Rad = MAX_RADIO
    r = 1
    b =self.Cell_Less(Id1,t+1,Rad)
    b.sort_values(by='AVAIL',inplace = True,ascending=False)
    a = b.iloc[:r,:]         #look for cells with radio less than rad.
    self.Dist_Mill = a.loc[:, ('DEST','DIST')]  #fill Distances to mills
    c = self.Disp_Radio(Id1, t+1,Rad)
    self.Available_Mill = c[:r]
    D1 = np.sum(self.Available_Mill)
    print('**Demanda:', Dem, 'Disponible',D1, 'Year: ',t, 'Mill: ',Id1, "Radio",Rad )  
    
    while (D1<=Dem):
      r +=1
      a = b.iloc[:r,:]         #look for cells with radio less than rad.
      #print('a:',a)
      self.Available_Mill = c[:r]
      self.Dist_Mill = a.loc[:, ('DEST','DIST')]  #fill Distances to mills
      D1 = np.sum(self.Available_Mill)
      print('**Demanda:', Dem, 'Disponible: ',D1, 'Year: ',t, 'Mill: ',Id1 )  
    
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    r1 = len(self.Available[0])
    Ha_li =[0]*r1
    self.By_Mill[Name_Mill] = Ha_li
        
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    if (mode == DISTANCE_MODE):
      prod0 = m.Intermediate(m.sum([(HAULING_COST+TRANSP_COST* self.Dist_Mill.loc[i,('DIST')])*H[i] for i in range(Num)]))    
    elif (mode ==DENSITY_MODE):
      prod0 = m.Intermediate(-m.sum([self.Available_Mill[i]*H[i] for i in range(Num)]))
    else:
      print("ERROR: Harvesting Mode Incorrect..")
 
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    flat = [item for l in H for item in l]
    a = self.Dist_Mill
    a['Harvested'] = flat  
    ans = a
    ans.reset_index(inplace = True)
    
    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested[t][self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[t].loc[a2,("Harvested")] = self.Harvested[t].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  
        self.By_Mill[Name_Mill][a2] = ans.loc[j,('Harvested')] 
      #else:
        #print(ans.loc[j,('DEST')], 'no hallado')
    
    #Updating Available Inventory
    for j in range(len(ans)):                                                                         
      if not(self.Available[t+1][self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[t+1].loc[a2,("SumOfTotal")] = self.Available[t+1].loc[a2,("SumOfTotal")] - int(ans.loc[j,('Harvested')]) 


  def Harvest_Refinery(self,t,Cand, Demanda=200000): #Evaluates Candidate location for refinery in year t
    m2 = GEKKO(remote=True)
    m2.options.solver = 3 #IPOPT solver
    m2.options.IMODE = 3 #steady state optimization
    print('Cand', Cand)
    
    Id1 =self.Candidates[Cand] #assign unique ID
    a =self.Cell_Less_Res(Id1,t,RADIO_REFINERY) #look for hexagons between radio distance
    self.Dist_Refi = a.loc[:, ('DEST','DIST')]  #obtain column Destination and Distance from 'a'
    Num = len(self.Dist_Refi) #calculate the number of cells betwee 
    H = m2.Array(m2.Var,(Num),value = 0)
    self.Available_Refi = self.Disp_Radio_Res(Id1, t+1, RADIO_REFINERY) #look for available quantity in cells
      
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Refi[i]  
    prod0 = m2.Intermediate(m2.sum([self.Dist_Refi.loc[i,('DIST')]*H[i] for i in range(Num)]))
    m2.Equation(Demanda ==m2.sum([H[i] for i in range(0,len(H))]))
    m2.Obj(prod0) 
    m2.solve(disp=False,debug=False)
    a = self.Dist_Refi
    flat = [item for l in H for item in l]
    a['Harvested'] = flat
    print('Sum of Harvested', np.sum(flat))
    ans = a
        
    #Storing Harvested Residues
    for j in range(len(ans)):                                                                         
      if not(self.Harvested_Res[t][self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested_Res[t].loc[a2,("Harvested")] = self.Harvested_Res[t].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  
    
    #Updating Available Inventory
    print('Residues Available Before',np.sum(self.AcumResidues[t+1].loc[:,("SumOfTotal")]))     
    
    for j in range(len(ans)):                                                                         
      if not(self.AcumResidues[t+1][self.AcumResidues[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.AcumResidues[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.AcumResidues[t+1].loc[a2,("SumOfTotal")] = self.AcumResidues[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
    
        l0  = self.AcumResidues[t+1]["SumOfTotal"]<0
        if np.sum(self.Available[t+1].loc[:,("SumOfTotal")])<0 :
          print('****Available Negative')
          
    
    print('answer\n',ans)
    print('Cells Harvested',self.Harvested_Res[t][self.Harvested_Res[t]["Harvested"]>0]) 
    print('Distances',self.Dist_Refi.loc[self.Harvested_Res[t]["Harvested"]>0],('DIST')) 
    print('Total Harvested',np.sum(self.Harvested_Res[t].loc[:,("Harvested")])) 
    print('Residues Available After',np.sum(self.AcumResidues[t+1].loc[:,("SumOfTotal")])) 
    res = round(np.dot(flat,self.Dist_Refi.loc[:,('DIST')]))  #Delivery cost to refinery
    self.RefCost[Cand] += res
    print('Cost:', self.RefCost[Cand])
 
  


**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [18]:
start_time = time.time()
# Main Program
YEARS = 1
 
#Upload distances from each hexagon to other
Hardwood_Saw_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Saw_V6.csv")
Hardwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Pulp_V6.csv")
Softwood_Saw_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Saw_V6.csv")
Softwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Pulp_V6.csv")
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Dist_Mill.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills3.csv")
Factor_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Roundwood_To_Residues.csv")

Growth_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Growth_Hexagon_V3.csv")
Growth_Data = Growth_Data[(Growth_Data['OWNERSHIP']=='PRIVATE')&(Growth_Data['SLOPE']=='LE40')] #FILTER OWNERSHIP AND SLOPE
 
i = Mills_Data[Mills_Data.Unique_ID==86399].index #removing this mills with no data
Mills_Data.drop(i, inplace=True)
 
print("Creating Inventories Objects and Initializing...")
Hardwood_Saw = Inventory(Hardwood_Saw_Data,Dist_Data,Wt='H',Pt ='Sawmill', Name = "Hardwood_Saw", Willingness = WILLING_FACTOR)
Hardwood_Pulp = Inventory(Hardwood_Pulp_Data,Dist_Data,Wt='H',Pt ='Pulpmill', Name = "Hardwood_Pulp", Willingness = WILLING_FACTOR)
Softwood_Saw = Inventory(Softwood_Saw_Data,Dist_Data,Wt='S',Pt ='Sawmill', Name = "Softwood_Saw", Willingness = WILLING_FACTOR)
Softwood_Pulp = Inventory(Softwood_Pulp_Data,Dist_Data,Wt='S',Pt ='Pulpmill', Name = "Softwood_Pulp", Willingness = WILLING_FACTOR)
Hardwood_Saw.Initialize_Totals(YEARS)

print("Updating Growth Rate...")
Hardwood_Saw.Growth_Update(Growth_Data)
Hardwood_Pulp.Growth_Update(Growth_Data)
Softwood_Saw.Growth_Update(Growth_Data)
Softwood_Pulp.Growth_Update(Growth_Data)
 
# Adjusting Mixed Mills, from a Mixed mill, two mills are created. a hardwood, and a softwood. 
#Demand is adjusted, so the 
Mixed = Mills_Data[Mills_Data['s_h']=='S/H'].copy()
for i in range(len(Mixed)):
  Hw = 0
  Sw = 0
  Tt = 0
  Id1 = Mixed.iloc[i,1]
  Id2 = Mills_Data[Mills_Data['Unique_ID']==Id1].copy()
  if Mills_Data.iloc[i,2] == "Sawmill" :
    Hw = Hardwood_Saw.Total_Available(Id1,20)
    Sw = Softwood_Saw.Total_Available(Id1,20)
  elif Mills_Data.iloc[i,2] == "Pulpmill" :
    Hw = Hardwood_Pulp.Total_Available(Id1,20)
    Sw = Softwood_Pulp.Total_Available(Id1,20)
  else:
    Sw = Softwood_Pulp.Total_Available(Id1,20)
    
  Tt = Hw + Sw
  Perc_Hw = Hw/Tt
  Id2['s_h'] ='S'
  Id2['Demand'] = Id2['Demand']*(1-Perc_Hw)
  Mills_Data.append(Id2)
  Mills_Data.loc[Mills_Data['Unique_ID']==Id1,('Demand')] = Mills_Data[Mills_Data['Unique_ID']==Id1].Demand*Perc_Hw
  Mills_Data.loc[Mills_Data['Unique_ID']==Id1,('s_h')] = 'H'
  #if Id1 == 90199 :
    #print('Hw',Hw)
    #print('Sw',Sw)
    #print('Tt',Tt)
    #print('Perc_Hw',Perc_Hw)
  
print('Updating Mills Data...')  
  
#updating Mills According the wood type and product type
Hardwood_Saw.Mills_Update(Mills_Data)
Hardwood_Pulp.Mills_Update(Mills_Data)
Softwood_Saw.Mills_Update(Mills_Data)
Softwood_Pulp.Mills_Update(Mills_Data)
 
print('Harvesting...')  
 
#Harvest Inventory Objects
Hardwood_Saw.Harvest_All_Mills_All_Years(YEARS,mode =DENSITY_MODE)
Hardwood_Pulp.Harvest_All_Mills_All_Years(YEARS,mode =DENSITY_MODE)
Softwood_Saw.Harvest_All_Mills_All_Years(YEARS,mode =DENSITY_MODE)
Softwood_Pulp.Harvest_All_Mills_All_Years(YEARS,mode =DENSITY_MODE)
 
Hardwood_Saw.Total_Harvested_Update(YEARS)
Hardwood_Pulp.Total_Harvested_Update(YEARS)
Softwood_Saw.Total_Harvested_Update(YEARS)
Softwood_Pulp.Total_Harvested_Update(YEARS)
 
Hardwood_Saw.Total_Residues_Update(YEARS)
Hardwood_Pulp.Total_Residues_Update(YEARS)
Softwood_Saw.Total_Residues_Update(YEARS)
Softwood_Pulp.Total_Residues_Update(YEARS)
 
print('TOTAL RESIDUES', np.sum(Hardwood_Saw.__class__.Total_Residues[YEARS].Harvested))

for y in range(YEARS+1):
  print('TOTAL HARVESTED at Hardwood_saw at',y,'  ', np.sum(Hardwood_Saw.Harvested[y].loc[:,('Harvested')]))
for y in range(YEARS+1):
  print('TOTAL HARVESTED at Hardwood_Pulp at',y,'  ', np.sum(Hardwood_Pulp.Harvested[y].loc[:,('Harvested')]))
for y in range(YEARS+1):
  print('TOTAL HARVESTED at Softwood_saw at',y,'  ', np.sum(Softwood_Saw.Harvested[y].loc[:,('Harvested')]))
for y in range(YEARS+1):
  print('TOTAL HARVESTED at softwood_Pulp at',y,'  ', np.sum(Softwood_Pulp.Harvested[y].loc[:,('Harvested')]))
 
for y in range(YEARS):
  Hardwood_Saw.__class__.Total_Harvested[y].to_csv('/content/drive/My Drive/Colab Notebooks/Results/TOTAL'+str(y)+'.csv')
 
 
print ("*********1ST STAGE TOOK", time.time() - start_time, "TO RUN")

TOTAL RESIDUES 3167351.0


**SECOND STAGE**

In [11]:
start_time = time.time()
ma = Dist_Data['ORIG'].unique()
Hardwood_Saw.Candidates = ma
Hardwood_Saw.Optimal_Location_Total(YEARS,2)

print ("*********2ND STAGE TOOK", time.time() - start_time, "TO RUN")

Cand 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Sum of Harvested 0.0
Residues Available Before 0
answer
        DEST   DIST  Harvested
0     40476   0.00        0.0
1     40283   2.95        0.0
2     40669   3.12        0.0
3     40089   3.32        0.0
4     40670   3.44        0.0
...     ...    ...        ...
2215  49422  99.72        0.0
2216  54549  99.78        0.0
2217  46073  99.80        0.0
2218  53347  99.83        0.0
2219  51978  99.97        0.0

[2220 rows x 3 columns]
Cells Harvested Empty DataFrame
Columns: [Unique_ID, Harvested]
Index: []
Distances Empty DataFrame
Columns: [DEST, DIST, Harvested]
Index: [] DIST
Total Harvested 0.0
Residues Available After 0.0
Cost: 0
Cand 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Sum of Harvested 0.0
Residues Available Before 0
answer
        DEST   DIST  Harvested
0     40669   0.00        0.0
1     40476   3.12        0.0
2     40862   3.13        0.0
3     40861   3.75        0.0
4     40282   3.80        0.0
...     ...    ...        ...
2277  47275  99.94        0.0
2278  50008  99.97        0.0
2279  54949  99.97        0.0
2280  55358  99.98        0.0
2281  46072  99.99        0.0

[2282 rows x 3 columns]
Cells Harvested Empty DataFrame
Columns: [Unique_ID, Harvested]
Index: []
Distances Empty DataFrame
Columns: [DEST, DIST, Harvested]
Index: [] DIST
Total Harvested 0.0
Residues Available After 0.0
Cost: 0
*********2ND STAGE TOOK 74.13367295265198 TO RUN
